# Default building control setting up an empty action interface

If you wish to run a simulation with all the default building controls (as specified in the building file), you can directly use the **EnergyPlus simulation engine**. For instance, from the container's workspace, you could run the following command:

```bash
$ energyplus -w sinergym/data/weather/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
```

However, doing this without our framework has some **drawbacks**. You'll only have the default EnergyPlus output and won't have access to some **additional outputs**, such as our wrapper logger that tracks all environment interactions. The buildings have a default *Site:Location* and *SizingPeriod:DesignDay*, which Sinergym automatically adjusts based on the specified weather, so you'd need to **modify** these before manually using the simulator. Lastly, the buildings' *RunPeriod* is set to one episode for DRL, meaning you can only simulate one year as it currently stands. Therefore, you'd also need to **manually** adjust the *RunPeriod* in the building file before starting the simulation.

Hence, we recommend setting up an **empty action interface** in a Sinergym environment. For instance:

In [1]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import LoggerWrapper

env = gym.make(
    'Eplus-office-hot-continuous-v1',
    actuators={},
    action_space=gym.spaces.Box(
        low=0,
        high=0,
        shape=(0,)))
env = LoggerWrapper(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    truncated = terminated = False
    current_month = 0
    while not (terminated or truncated):
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [office-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 6, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward functi

/usr/local/lib/python3.12/dist-packages/opyplus/weather_data/weather_data.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1/Eplus-env-sub_run1/ASHRAE901_OfficeMedium_STD2019_Denver.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
Reward:  -3.400918408046664 {'time_elapsed(hours)': 0.5, 'month': 1, 'day': 1, 'hour': 0, 'is_raining': False, 'action': array([], dtype=float32), 'timestep': 2, 'reward': -3.400918408046664, 'energy_term': -0.008430450234156376, 'comfort_term': -3.3924879578125076, 'reward_weight': 0.5, 'abs_energy_penalty': -168.6090046831275, 'abs_comfort_penalty': -6.784975915625015, 'total_power_demand': 168.609

In this example, a default environment is loaded, but the space and definition of the default action **are replaced with an empty one**. Sinergym handles the necessary background changes. The implemented random agent then sends **empty actions ([])**.

The benefits of this approach include the ability to **mix and match weathers and buildings** as desired, with Sinergym automatically making all necessary adjustments (avoiding the previously mentioned disadvantages). You can run as many years as you want in a single experiment, using our loggers. You also have more flexibility in choosing which observation variables you want. When you set an empty action interface, Sinergym retains the default actuators that come with the building definition (these can vary in sophistication depending on the building definition in the *epJSON* file).